In [34]:
# utilizado para la manipulación de directorios y rutas
import os

# Cálculo científico y vectorial para python
import numpy as np

# Libreria para graficos
from matplotlib import pyplot

# Modulo de optimizacion en scipy
from scipy import optimize
import pandas as pd

# modulo para cargar archivos en formato MATLAB
# from scipy.io import loadmat

# le dice a matplotlib que incruste gráficos en el cuaderno
%matplotlib inline

1. Preparamos los datos

En esta parte estaremos cargando el data set y dividiendo los datos en entrenamiento y prueba, de igual forma la columna "personality_type" es un string y los volverelos a enteros con la siguiente clasificacion

Introvert = 0

Extrovert = 1

Ambivert = 2

In [35]:
# La entrada es de 29 elemento contando con x0
input_layer_size  = 29

# 10 etiquetas, de 1 a 10 (tomar en cuenta que se asigna "0" a la etiqueta 10)
num_labels = 3

#  datos de entrenamiento almacenados en los arreglos X, y
# data = loadmat(os.path.join('Datasets', 'ex3data1.mat'))
data = pd.read_csv('personalidad.csv', delimiter=',')

# Mapear y a valores enteros
mapping = {'Introvert': 0, 'Extrovert': 1, 'Ambivert': 2}
data["personalidad"] = data["personality_type"].map(mapping)

# Definir proporción
train_size = int(0.8 * len(data))  # 16000 si hay 20000 filas

# División en train y test
train_data = data[:train_size]   # primeras 16000 filas
test_data = data[train_size:]    # últimas 4000 filas

X = train_data.drop(columns=["personality_type", "personalidad"])  # todas menos las columnas personality_type y personalidad
y = train_data["personalidad"]

m = y.size
print("Valores de X:")
print(X)

print("Valores de y:")
print(y)

Valores de X:
       social_energy  alone_time_preference  talkativeness  deep_reflection  \
0           6.794295               3.854670       8.725446         2.515151   
1           6.378988               5.731157       7.029529         7.274493   
2           7.459421               6.322263       3.922269         4.622261   
3           6.159626               3.097837       6.019093         1.965440   
4           5.568462               6.986722       3.913240         9.926161   
...              ...                    ...            ...              ...   
15995       1.595353               9.384099       2.583868         9.415669   
15996       7.361423               4.858535       8.759608         1.458982   
15997       5.628355               6.455090       4.912531         5.718575   
15998       2.819564               5.503792       5.002545         4.260116   
15999       6.438485               5.112357       5.746069         4.430790   

       group_comfort  party_liking  l

2. Verificamos que no haya datos nulos

In [36]:
print(X.isnull().sum())

social_energy               0
alone_time_preference       0
talkativeness               0
deep_reflection             0
group_comfort               0
party_liking                0
listening_skill             0
empathy                     0
creativity                  0
organization                0
leadership                  0
risk_taking                 0
public_speaking_comfort     0
curiosity                   0
routine_preference          0
excitement_seeking          0
friendliness                0
emotional_stability         0
planning                    0
spontaneity                 0
adventurousness             0
reading_habit               0
sports_interest             0
online_social_usage         0
travel_desire               0
gadget_usage                0
work_style_collaborative    0
decision_speed              0
stress_handling             0
dtype: int64


In [37]:
print(y.isnull().sum())

0


3. Funcion de normalizacion tomado del cuadernillo vistp en clases

In [38]:
def  featureNormalize(X):
    X_norm = X.copy()
    mu = np.zeros(X.shape[1])
    sigma = np.zeros(X.shape[1])

    mu = np.mean(X, axis = 0)
    sigma = np.std(X, axis = 0)
    X_norm = (X - mu) / sigma

    return X_norm, mu, sigma

In [39]:
# llama featureNormalize con los datos cargados
X_norm, mu, sigma = featureNormalize(X)

Vemos como fue la normalizacion de los datos tomando la primera fila de cada columna

In [40]:
# Primera fila como Serie de pandas
print(X_norm.iloc[0])

# Primera fila como array de NumPy
print(X_norm.iloc[0].values)


social_energy               0.522610
alone_time_preference      -0.648689
talkativeness               1.289322
deep_reflection            -1.592060
group_comfort               0.502980
party_liking                1.268475
listening_skill            -0.117429
empathy                    -0.034837
creativity                  0.100619
organization               -3.511566
leadership                  1.142187
risk_taking                 0.863299
public_speaking_comfort     1.265751
curiosity                  -0.639998
routine_preference         -0.632569
excitement_seeking          1.361767
friendliness                0.846167
emotional_stability        -0.740958
planning                   -0.925617
spontaneity                -0.328033
adventurousness             1.463648
reading_habit              -0.316225
sports_interest             2.342861
online_social_usage         1.402793
travel_desire              -0.350426
gadget_usage                1.892853
work_style_collaborative    0.963197
d

In [41]:
# Configurar la matriz adecuadamente, y agregar una columna de unos que corresponde al termino de intercepción.
m, n = X.shape
# Agregar el término de intercepción
X = X_norm
#X = np.concatenate([np.ones((m, 1)), X], axis=1)
print(X)

       social_energy  alone_time_preference  talkativeness  deep_reflection  \
0           0.522610              -0.648689       1.289322        -1.592060   
1           0.354847               0.103499       0.609493         0.599112   
2           0.791286               0.340443      -0.636090        -0.621959   
3           0.266237              -0.952065       0.204447        -1.845143   
4           0.027437               0.606791      -0.639710         1.819923   
...              ...                    ...            ...              ...   
15995      -1.577492               1.567776      -1.172605         1.584896   
15996       0.751700              -0.246291       1.303016        -2.078313   
15997       0.051631               0.393687      -0.239132        -0.117223   
15998      -1.082974               0.012360      -0.203048        -0.788689   
15999       0.378881              -0.144547       0.095002        -0.710111   

       group_comfort  party_liking  listening_skill

In [42]:
def sigmoid(z):
    """
    Calcula la sigmoide de z.
    """
    return 1.0 / (1.0 + np.exp(-z))

In [43]:
def calcularCosto(theta, X, y):
    # Inicializar algunos valores utiles
    m = y.size  # numero de ejemplos de entrenamiento

    J = 0
    h = sigmoid(X.dot(theta.T))
    J = (1 / m) * np.sum(-y.dot(np.log(h)) - (1 - y).dot(np.log(1 - h)))

    return J

In [44]:
def descensoGradiente(theta, X, y, alpha, num_iters):
    # Inicializa algunos valores
    m = y.shape[0] # numero de ejemplos de entrenamiento

    # realiza una copia de theta, el cual será acutalizada por el descenso por el gradiente
    theta = theta.copy()
    J_history = []

    for i in range(num_iters):
        h = sigmoid(X.dot(theta.T))
        theta = theta - (alpha / m) * (h - y).dot(X)

        J_history.append(calcularCosto(theta, X, y))
    return theta, J_history

In [45]:
def lrCostFunction(theta, X, y, lambda_):
    """
    Calcula el costo de usar theta como parámetro para la regresión logística regularizada y
    el gradiente del costo w.r.t. a los parámetros.

    Parametros
    ----------
    theta : array_like
        Parametro theta de la regresion logistica. Vector de la forma(shape) (n, ). n es el numero de caracteristicas
        incluida la intercepcion

    X : array_like
        Dataset con la forma(shape) (m x n). m es el numero de ejemplos, y n es el numero de
        caracteristicas (incluida la intercepcion).

    y : array_like
        El conjunto de etiquetas. Un vector con la forma (shape) (m, ). m es el numero de ejemplos

    lambda_ : float
        Parametro de regularización.

    Devuelve
    -------
    J : float
        El valor calculado para la funcion de costo regularizada.

    grad : array_like
        Un vector de la forma (shape) (n, ) que es el gradiente de la
        función de costo con respecto a theta, en los valores actuales de theta..
    """
#     alpha = 0.003
#     theta = theta.copy()
    # Inicializa algunos valores utiles
    m = y.size

    # convierte las etiquetas a valores enteros si son boleanos
    if y.dtype == bool:
        y = y.astype(int)

    J = 0
    grad = np.zeros(theta.shape)  # Inicializa el gradiente para theta (esta en filas)

    h = sigmoid(X.dot(theta.T))   #transpuesta de theta   (ahora con la transpuesta estara en columnas)

    temp = theta
    temp[0] = 0

#     J = (1 / m) * np.sum(-y.dot(np.log(h)) - (1 - y).dot(np.log(1 - h)))
    J = (1 / m) * np.sum(-y.dot(np.log(h)) - (1 - y).dot(np.log(1 - h))) + (lambda_ / (2 * m)) * np.sum(np.square(temp))

    grad = (1 / m) * (h - y).dot(X)
#     theta = theta - (alpha / m) * (h - y).dot(X)
    grad = grad + (lambda_ / m) * temp

    return J, grad
#    return J, theta

In [46]:
def OneVsAll(X, y, num_labels, lambda_):
  alpha = 0.001
  num_iters = 100000

  m, n = X.shape
  all_theta = np.zeros((num_labels, n + 1))

  # Agrega unos a la matriz X
  X = np.concatenate([np.ones((m, 1)), X], axis=1)

  for c in np.arange(num_labels):
      initial_theta = np.zeros(n + 1)

      y_actual = np.where(y == c, 1, 0)

      theta, J_history = descensoGradiente(initial_theta, X, y_actual, alpha, num_iters)

      all_theta[c] = theta
      # Grafica la convergencia del costo
      pyplot.plot(np.arange(len(J_history)), J_history, lw=2)
      pyplot.xlabel('Numero de iteraciones')
      pyplot.ylabel('Costo J')

  return all_theta

In [47]:
def OneVsAllOM(X, y, num_labels, lambda_):
    """
    Trains num_labels logistic regression classifiers and returns
    each of these classifiers in a matrix all_theta, where the i-th
    row of all_theta corresponds to the classifier for label i.

    Parameters
    ----------
    X : array_like
        The input dataset of shape (m x n). m is the number of
        data points, and n is the number of features. Note that we
        do not assume that the intercept term (or bias) is in X, however
        we provide the code below to add the bias term to X.

    y : array_like
        The data labels. A vector of shape (m, ).

    num_labels : int
        Number of possible labels.

    lambda_ : float
        The logistic regularization parameter.

    Returns
    -------
    all_theta : array_like
        The trained parameters for logistic regression for each class.
        This is a matrix of shape (K x n+1) where K is number of classes
        (ie. `numlabels`) and n is number of features without the bias.
    """
    # algunas variables utiles
    m, n = X.shape

    all_theta = np.zeros((num_labels, n + 1))

    # Agrega unos a la matriz X
    X = np.concatenate([np.ones((m, 1)), X], axis=1)

    for c in np.arange(num_labels):
        initial_theta = np.zeros(n + 1)
        options = {'maxiter': 10}
        res = optimize.minimize(lrCostFunction,
                                initial_theta,
                                (X, (y == c), lambda_),
                                jac=True,
                                method='CG',
                                options=options)

        all_theta[c] = res.x


    return all_theta

In [48]:
lambda_ = 0.1  # coeficiente de regularización
all_theta = OneVsAllOM(X, y, num_labels, lambda_)
print(all_theta.shape)

(3, 30)


In [49]:
print(all_theta)

[[-1.11321945e+01 -1.42816357e+00  1.99479344e+00 -1.60247550e+00
   1.31213785e+00 -1.19926047e+00 -2.35433672e+00  6.87254668e-01
   2.64170098e-01 -9.07067687e-03  1.73010812e-01 -8.10338873e-01
  -9.47909008e-01 -1.72160321e+00 -1.23217157e-02  1.00652279e+00
  -1.62534258e+00 -3.68058435e-01 -1.72795200e-01  6.57267789e-01
  -5.52412176e-01 -1.25997229e+00  1.06817457e+00 -5.77011696e-01
  -9.58692503e-01 -7.81210266e-01 -4.87843202e-01 -5.08369773e-01
  -7.19364516e-01  2.65191789e-01]
 [-1.31332642e+01  1.56926228e+00 -1.84492685e+00  1.91714012e+00
  -1.49253554e+00  5.32089122e-01  2.60247379e+00 -3.34948244e-01
  -5.90916287e-01 -2.09174909e-01 -9.98440642e-01  1.02781219e+00
   1.44834966e+00  1.53271784e+00  6.17773286e-01 -1.05256356e+00
   1.71924280e+00  6.58356102e-01  1.22864128e-02 -7.06459333e-01
   8.44677109e-01  1.91398446e+00 -1.08175307e+00  1.53324752e+00
   4.15076244e-01  1.43067514e+00  6.07802865e-01  9.51951520e-01
   1.48837724e+00 -1.93227819e-01]
 [-7.3

In [50]:
def predictOneVsAll(all_theta, X):
    """
    Devuelve un vector de predicciones para cada ejemplo en la matriz X.
    Tenga en cuenta que X contiene los ejemplos en filas.
    all_theta es una matriz donde la i-ésima fila es un vector theta de regresión logística entrenada para la i-ésima clase.
    Debe establecer p en un vector de valores de 0..K-1 (por ejemplo, p = [0, 2, 0, 1]
    predice clases 0, 2, 0, 1 para 4 ejemplos).

    Parametros
    ----------
    all_theta : array_like
        The trained parameters for logistic regression for each class.
        This is a matrix of shape (K x n+1) where K is number of classes
        and n is number of features without the bias.

    X : array_like
        Data points to predict their labels. This is a matrix of shape
        (m x n) where m is number of data points to predict, and n is number
        of features without the bias term. Note we add the bias term for X in
        this function.

    Devuelve
    -------
    p : array_like
        The predictions for each data point in X. This is a vector of shape (m, ).
    """

    m = X.shape[0]
    num_labels = all_theta.shape[0]

    p = np.zeros(m)

    # Add ones to the X data matrix
    X = np.concatenate([np.ones((m, 1)), X], axis=1)
    p = np.argmax(sigmoid(X.dot(all_theta.T)), axis = 1)

    return p

In [51]:
print(X.shape)
pred = predictOneVsAll(all_theta, X)
print('Precision del conjuto de entrenamiento: {:.2f}%'.format(np.mean(pred == y) * 100))
test_data=test_data.drop(columns=["personality_type", "personalidad"])
print(test_data)
XPrueba = test_data.copy()
print(XPrueba.shape)

print(XPrueba.shape)
#print(np.ones((1)))
#print(XPrueba)
#p = np.zeros(1)
XPrueba = np.concatenate([np.ones((XPrueba.shape[0], 1)), XPrueba], axis=1)
print(XPrueba.shape)
p = np.argmax(sigmoid(XPrueba.dot(all_theta.T)), axis = 1)
print(p)

# displayData(X[1002:1003, :])
print(y[16000:19999])

(16000, 29)
Precision del conjuto de entrenamiento: 99.83%
       social_energy  alone_time_preference  talkativeness  deep_reflection  \
16000       3.129031               5.596797       3.646650         7.039760   
16001       4.500816               8.221506       5.282489         7.565395   
16002       8.912206               0.844801       8.659337         3.165870   
16003       3.179922              10.000000       5.027274         4.037840   
16004       4.966248               3.951927       5.354487         6.384211   
...              ...                    ...            ...              ...   
19995       3.783857               4.345803       8.220063         6.297798   
19996       2.621406               8.152097       2.954341         8.791467   
19997       0.039689               8.277160       5.728101         7.790080   
19998       7.756725               3.956191       8.906972         3.793775   
19999       3.225794               8.058594       0.000000        10.000